In [1]:
import pandas as pd
import random
from random import randint
from faker import Faker
from pandas_datareader import data as web
from yahoo_fin.stock_info import *
import numpy as np

In [2]:
model_holding = pd.read_csv('model_holdings.csv', index_col=0)

In [3]:
from faker.providers import BaseProvider


class StrategyProvider(BaseProvider):

    __provider__ = 'strategy'
    __lang___ = 'en_US'

    strategy_name = ['Income','Aggressive','Capital Appreciation','Speculation']
    prim_fin_need = ['Wealth Accumulation/Investment','Retirement','Major Purchase/Expense','Education Planning',
                     'Current Income','Health Care/Long Term Care','Estate/Legacy Planning','Charitable']
    inv_horizon = ['Immediate','Less than 2 Years','2 to 5 Years','6 to 10 Years','11 to 20 Years','Over 20 Years']
    liquidity_need = ['Lump Sum','Less than 2 Years','2 to 5 Years','6 to 10 Years','11 to 20 Years','Over 20 Years']
    #invest_obj = []
    #risk_tolerance = []

    def strategyName(self):
        return self.random_element(self.strategy_name)
    def investObj(self, invest_obj):
        return self.random_element(invest_obj)
    def riskTolerance(self, risk_tolerance):
        return self.random_element(risk_tolerance)
    def primFinNeed(self):
        return self.random_element(self.prim_fin_need)
    def invHorizon(self):
        return self.random_element(self.inv_horizon)
    def liquidityNeed(self):
        return self.random_element(self.liquidity_need)

In [4]:
fake = Faker()
fake.add_provider(StrategyProvider)

In [16]:
def computeModelVolatility(model_id):
    holding = model_holding[model_holding.model_id.eq(model_id)]
    price = pd.DataFrame()
    start_date = '01/01/2019'
    end_date = '01/01/2020'
    
    for stock in holding.product_id.unique().tolist():
        try:
            price[stock] = get_data(stock , start_date , end_date)['adjclose'].dropna()
        except Exception:
            index = pd.date_range(start_date , end_date)
            price[stock] = pd.DataFrame(index=index, columns=['adjclose'])
            price[stock] = price.fillna(0.0)

    d_returns = price.pct_change()  
    cov_matrix_d = d_returns.cov()
    cov_matrix_a = cov_matrix_d * 252
        
    weights = holding[['percent']].to_numpy() # get weights
    weights = weights/100

    # calculate the variance and risk of the portfolo
    port_variance = np.dot(weights.T, np.dot(cov_matrix_a, weights))
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix_a, weights)))
    
    percent_var = (np.round(port_variance, 4) * 100)
    percent_vols = (np.round(port_volatility, 4) * 100)
    
    return percent_vols.item(0)

In [17]:
computeModelVolatility(3019)

0.0

In [18]:
cols=['model_id','model_name','account_name','volatility','risk_profile','investment_amt','inv_obj_most_imp','inv_obj_very_imp',
      'inv_obj_some_imp','inv_obj_least_imp','risk_tolerance','primary_fin_need','inv_horizon','liquidity_need']
models = pd.DataFrame(columns=cols) 
myList = []
i = 1;
for model_id in model_holding.model_id.unique().tolist():
    #invest_obj = ['Income','Aggressive Income','Capital Appreciation','Speculation']
    invest_obj_dict = {
        1:'Income',
        2:'Aggressive Income',
        3:'Capital Appreciation',
        4:'Speculation'
    }
    invest_obj = [1,2,3,4]
    invest_obj_most_imp = fake.investObj(invest_obj)
    invest_obj.remove(invest_obj_most_imp)
    invest_obj.append(0)

    invest_obj_very_imp = fake.investObj(invest_obj)
    invest_obj.remove(invest_obj_very_imp)
    invest_obj.append(0)
    
    invest_obj_some_imp = fake.investObj(invest_obj)
    invest_obj.remove(invest_obj_some_imp)
    invest_obj.append(0)
    
    invest_obj_least_imp = fake.investObj(invest_obj)
    #risk_tolerance = ['Moderate','Aggressive']
    risk_tolerance = [1,2]
    if(invest_obj_most_imp == 5):
        risk_tolerance.remove(1)
        risk_profile = 5
    if(invest_obj_most_imp == 1):
        risk_tolerance.remove(2)
        risk_tolerance.append(0)
    risk_tol = fake.riskTolerance(risk_tolerance)
    
    #=SUM(D2*$B$2/4,E2*$B$3/4/4,H2*2/3)
    
    risk_profile = (invest_obj_most_imp * 5/4) + (invest_obj_very_imp * 1/4) + (risk_tol * 2/3);
    
    risk_profile = int(risk_profile)
    
    volatility = computeModelVolatility(model_id)
    
    myList.append([model_id,invest_obj_dict[invest_obj_most_imp] + " Strategy " + str(i),fake.name(), volatility, risk_profile, 
                   randint(10000,1000000), invest_obj_most_imp,invest_obj_very_imp,invest_obj_some_imp,invest_obj_least_imp,
                   risk_tol,fake.primFinNeed(),fake.invHorizon(),fake.liquidityNeed()])
    i = i + 1
df = pd.DataFrame(myList, columns=cols)
models=models.append(df, sort=True)
models.to_csv('models.csv')

ValueError: shapes (11,11) and (12,1) not aligned: 11 (dim 1) != 12 (dim 0)